In [1]:
access_token = 'EAAH24PQU4egBADGV6Xd8EnwaCmEjvlKZCLYsymsSZBVCPBvvZBtcOnMuotunZAGh4BBvAP9dHYLAnS5J70UOIXDgEbxhgHbGXOTWGAESBfSlJGj6GDZAlDvh5KYzAztgsaOxbGSlVLaT29P00gmsSLBNGXiSZBT71mRUa5rsFo2I8ZC6d4I0oBFFZCaqbyKbbCMZD'
host = '78.46.226.67'
port = '5432'
database ='postgres'
user='kirill_k'
password='swfr35$#'

address = 'postgresql://' + user +':' + password + '@' + host +':' + port + '/' + database

In [2]:
import psycopg2
!pip install facebook-business
from tqdm import tqdm

import pandas as pd
import numpy as np

from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adreportrun import AdReportRun
from facebook_business.exceptions import FacebookRequestError, FacebookBadObjectError
from facebook_business.adobjects.user import User
from facebook_business.api import FacebookAdsApi
from psycopg2 import OperationalError

import warnings

import json

import datetime
from datetime import timedelta, datetime, date, time
import time

import requests as rq


/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


     |████████████████████████████████| 1.0MB 2.8MB/s 
     |████████████████████████████████| 10.1MB 12.2MB/s 
  Created wheel for curlify: filename=curlify-2.2.1-cp36-none-any.whl size=2660 sha256=c2402dfbd5fe72d6fe8c983dc1f8ff185d330242a91d2c3e21c87838c982ac4a
  Stored in directory: /root/.cache/pip/wheels/b7/5e/e5/6f90a2b21faf7b71748fcba58920ad77850c25b8f70927d418
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746863 sha256=88019ee9ad0a8296ab30f3754474c706f49fc1901d2fef38b065ab1734d0592b
  Stored in directory: /root/.cache/pip/wheels/33/4e/a6/be297e6b83567e537bed9df4a93f8590ec01c1acfbcd405348
Successfully built curlify pycountry


In [3]:
def connection_to_database(host, port, database, user, password):
    connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
    cur = connection.cursor()
    return cur, connection

def reconnection_to_database(host, port, database, user, password, connection):
    if connection.closed > 0:
        connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
        cur = connection.cursor()
    else:
        try:
            connection.cursor().execute('SELECT 1')
        except OperationalError:
            connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
            cur = connection.cursor()
    return connection.cursor(), connection

def list_days(start_date, end_date):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    d = timedelta(1)
    date_list = []
    while start_date != end_date + d:
        date_list.append(start_date)
        start_date = start_date + d

    return date_list

def dict2json(dictionary):
    return json.dumps(dictionary, ensure_ascii=False, sort_keys=True)

def modify_object(object_data):
    for key in object_data.keys():
        if type(object_data[key]) in (list, dict):
            object_data[key] = dict2json(object_data[key])
    
    return object_data

def store_data(data, table):
    for keyword in data:
        fields = ','.join(keyword.keys())
        values = ','.join(map(lambda x:f"%({x})s",keyword.keys()))
        query =   f"INSERT INTO {table} ({fields}) values ({values})"
        cur.execute(query, keyword)


In [4]:
#Function to check how close you are to the FB Rate Limit
def check_limit(account_id, version = '6.0'):
    check=rq.get('https://graph.facebook.com/v' + version + '/act_'+account_id+'/insights?access_token='+access_token)
    usage = check.headers['x-fb-ads-insights-throttle']
    return json.loads(usage)

In [5]:
FacebookAdsApi.init(access_token=access_token)

delta = 30
level = 'ad'
action_attribution_windows = ['1d_view', '7d_view', '28d_view','1d_click', '7d_click', '28d_click']

yesterday =  (datetime.utcnow() - timedelta(days = 1)).strftime('%Y-%m-%d')
start_date = (datetime.utcnow() - timedelta(days = delta)).strftime('%Y-%m-%d')
                                    
try:
    cur.close()
    cur, connection = connection_to_database(host, port, database, user, password)
except NameError:
    cur, connection = connection_to_database(host, port, database, user, password)

me = User(fbid='me')
id_list = list(me.get_ad_accounts())
id_list = list(map(lambda x:x.export_all_data(), id_list))
id_list = list(map(lambda x:x['account_id'], id_list))
id_list = tuple(id_list)
    
table = 'insights'
table_log = 'insights_log'

cur.execute(f"""CREATE TABLE if not exists {table_log}(
    account_id VARCHAR(50),     date date,     with_data BOOLEAN,
    recording_date timestamp without time zone,
    PRIMARY KEY (account_id, date))""")
connection.commit()

cur.execute(f""" CREATE TABLE if not exists {table}(
    account_id VARCHAR(50),     
    campaign_id VARCHAR(50),     adset_id VARCHAR(50), 
    ad_id VARCHAR(50),     account_currency VARCHAR(10),     date_start DATE,
    date_stop DATE,    buying_type VARCHAR(20),    impressions BIGINT,
    reach BIGINT,    clicks BIGINT,    spend NUMERIC,    social_spend NUMERIC,
    unique_inline_link_clicks BIGINT,  
    inline_link_clicks BIGINT,    inline_post_engagement BIGINT,    estimated_ad_recallers BIGINT,
    estimated_ad_recall_rate NUMERIC,    canvas_avg_view_percent NUMERIC,    canvas_avg_view_time NUMERIC,
    objective VARCHAR(50),    unique_clicks BIGINT,    relevance_score JSON,    actions JSONB,
    action_values JSONB,    outbound_clicks JSONB,     unique_actions JSONB,    unique_outbound_clicks JSONB,
    mobile_app_purchase_roas JSONB,    website_purchase_roas JSONB,    video_p25_watched_actions JSONB,
    video_p50_watched_actions JSONB,    video_p75_watched_actions JSONB,    video_p95_watched_actions JSONB,
    video_p100_watched_actions JSONB,    video_avg_percent_watched_actions JSONB,    video_10_sec_watched_actions JSONB,
    video_30_sec_watched_actions JSONB,    video_avg_time_watched_actions JSONB,
    video_play_actions JSONB,
    PRIMARY KEY (account_id, campaign_id, adset_id, ad_id, date_start),
    FOREIGN KEY (account_id, date_start) REFERENCES {table_log}(account_id, date) ON DELETE CASCADE)""")
connection.commit()

# корректно написать запрос на выборку полей
# fields = list(AdKeywordStats.Field.__dict__.keys())
# exclude_list = ['__module__', '__doc__']
#fields = list(set(fields)-set(exclude_list ))

fields = ['account_id', 'campaign_id', 'adset_id', 'ad_id', 'date_start', 'date_stop', 'impressions', 'reach',
          'clicks', 'spend',  'unique_inline_link_clicks',  
    'inline_link_clicks',    'inline_post_engagement',    'estimated_ad_recallers',
    'estimated_ad_recall_rate',  'objective',    'unique_clicks',    'actions',
    'action_values',    'outbound_clicks',     'unique_actions',    'unique_outbound_clicks']

days = tuple(map(lambda x:x.strftime('%Y-%m-%d'), list_days(start_date, yesterday)))
days = tuple(sorted(list(days), reverse=False))


for day in tqdm(days):
   
    query = f"""SELECT account_id FROM {table_log} where date = %s and account_id in %s""" 
    account_in_db = pd.read_sql(query, connection, params = (day, id_list)).account_id
    accounts = list(set(id_list) - set(account_in_db))

    for account_id in accounts:
        
        params = { 'level': level, 'time_range': {'since':day,'until':day}, 
                          'action_attribution_windows' : action_attribution_windows, 'fields': fields}
        try:
            if check_limit(account_id)['app_id_util_pct'] < 69 and check_limit(account_id)['acc_id_util_pct'] < 100:
              
            
                async_job = AdAccount('act_' + account_id).get_insights(params=params, is_async=True)

                ############# кусок для асинхронного получения
                async_job.api_get()                       
                percent = AdReportRun.Field.async_percent_completion
                status = AdReportRun.Field.async_status

                counter = 0

                while (counter < 100) and ((async_job[percent] < 100) or (async_job[status] != "Job Completed")):
                    time.sleep(1)
                    async_job.api_get()
                    if (async_job[percent] == 0) and (async_job[status] == "Job Not Started"):
                        counter = counter + 1

                time.sleep(1)

                if counter >= 100:
                    async_job = AdAccount('act_' + account_id).get_insights(fields=fields, params=params, is_async=False)
                    data = list(async_job)
                else:
                    data = list(async_job.get_result()) 
                ############# кусок для асинхронного получения

                recording_date = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
                with_data = len(data) > 0
                parameters  =  (account_id, day, with_data, recording_date)
                query_log = f"INSERT INTO {table_log} VALUES (%s,%s, %s, %s);"
                cur.execute(query_log, vars = parameters)

                if len(data) > 0:           
                    data = list(map(lambda x:x.export_all_data(),data))
                    tuple(map(lambda x:modify_object(x),data))
                    data = tuple(data)
                    store_data(data, table)

                connection.commit()
            else:
                print('append')
                time.sleep(30)
                accounts.append(account_id)

        except FacebookRequestError as error:
            print(check_limit(account_id))
            print(error)
            code = error._body['error']['code']
            subcode = error._body['error']['error_subcode']
            if code == 4 and subcode == 1504022:
                print(f'FacebookRequestError, sleeping {20} seconds')
                time.sleep(20)

  

100%|██████████| 30/30 [00:08<00:00,  3.61it/s]
